# Transfer Learning

Reference: [PyTorch documnetation](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [4]:
import torch
from torch import nn, optim
from torchvision import models, datasets, transforms
from torch.optim import lr_scheduler
import torch.nn.functional as F
import os

In [54]:
data_dir = "Cat_Dog_data"

imagenet_stats = [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]

data_transforms = {'train': transforms.Compose([
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(*imagenet_stats)]),
                   'test': transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize(*imagenet_stats)])
                  }

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                 for x in ['train','test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],batch_size=16,shuffle=True,num_workers=2) 
              for x in ['train','test']}

dataset_sizes = {x: len(dataloaders[x]) for x in ['train','test']}

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
num_classes = len(image_datasets['train'].classes)
image_datasets['train'].classes

['cat', 'dog']

In [56]:
model = models.resnet18(pretrained=True)
nf = model.fc.in_features
model.fc = nn.Linear(nf,num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)
scheduler = lr_scheduler.StepLR(optimizer,5,gamma=0.1)

In [57]:
from tqdm import tqdm

# Helper Classes

In [80]:
import pandas as pd
from IPython.display import display, HTML, clear_output

class Logger:
    def __init__(self):
        self.logs = []
    def append(self,current):
        self.logs.append(current)
        
    def show(self):
        clear_output(wait=True)
        df = pd.DataFrame(self.logs)
        display(HTML(df.to_html()))

class Trainer:
    def __init__(self,dls,model,loss_func,optimizer,scheduler=None):
        self.dls,self.model = dls, model
        self.optimizer,self.scheduler = optimizer,scheduler
        self.criterion = loss_func
        self.ds_sizes = {x: len(dls[x]) for x in ['train','test']}
        self.logger = Logger()
    
    def fit(self,epochs,steps=None):
        self.model.train()        
        for e in range(epochs):
            running_loss = 0.
            running_accuracy = 0.
            for idx, (images, labels) in enumerate(self.dls['train']):
                if idx+1 == steps:
                    break
                
                images, labels = images.to(device), labels.to(device)
                
                self.optimizer.zero_grad()
                
                with torch.set_grad_enabled(True):
                    outputs = self.model(images)
                    loss = self.criterion(outputs,labels)
                    
                    loss.backward()
                    self.optimizer.step()
                    if self.scheduler is not None:
                        self.scheduler.step()
                    
                    running_loss += loss.item()
                    correct = outputs.argmax(-1)==labels[None]
                    running_accuracy += torch.mean(correct.type(torch.FloatTensor)).item()
            
            test_logs = self.evaluate(steps)
            
            div_factor = self.ds_sizes['train'] if steps is None else steps
            
            logs = {'epoch':e,
                    'train_loss':running_loss/div_factor,
                    'train_accuracy':running_accuracy/div_factor,
                     **test_logs}
            
            self.logger.append(logs)
            self.logger.show()
    
    def evaluate(self,steps=None):
        "Evaluate on test dataset"
        self.model.eval()
        with torch.no_grad():
            running_loss = 0.
            running_accuracy = 0.
            for idx, (images,labels) in enumerate(self.dls['test']):
                if steps is not None and idx+1==steps:
                    break
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                loss = self.criterion(outputs,labels)
                correct = outputs.argmax(-1)==labels
                
                running_loss += loss.item()
                running_accuracy += torch.mean(correct.type(torch.FloatTensor)).item()
            

            div_factor = self.ds_sizes['test'] if steps is None else steps
            
            logs = {'test_loss': running_loss / div_factor,
                   'test_accuracy': running_accuracy / div_factor}
            
            return logs

In [83]:
trainer = Trainer(dataloaders,model,nn.CrossEntropyLoss(),optimizer,scheduler)

In [84]:
trainer.fit(20,steps=10)

,epoch,train_loss,train_accuracy,test_loss,test_accuracy
0,0,0.549134,0.76875,0.294270,0.83750
1,1,0.812275,0.68125,0.373479,0.81250
2,2,0.400133,0.78750,0.364354,0.80625
3,3,0.583836,0.75000,0.374108,0.80625
4,4,0.363689,0.78750,0.303404,0.81250
5,5,0.775697,0.70000,0.288825,0.83125
6,6,0.517143,0.76250,0.354549,0.80625
7,7,0.634965,0.71250,0.400485,0.81875
8,8,0.652000,0.73750,0.328406,0.81875
9,9,0.367845,0.78125,0.415547,0.80000
